# SMA - Optimal Strategy

Purpose: Optimize Strategy Moving Average values

In [17]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from itertools import product
from IPython.display import display, clear_output

In [18]:
class IterativeBase():
    def __init__(self):
        pass

    def get_data(self):
        raw = pd.read_csv("https://algo.kimbly.in/DataSets/nifty.csv", parse_dates=["date"], index_col="date")
        self.original_data = raw.copy()

    def initiate_values(self, ema_short, ema_long):
        self.data = self.original_data.copy(deep=True)
        self.data['EMA_S'] = ta.ema(self.data['close'], ema_short)
        self.data['EMA_L'] = ta.ema(self.data['close'], ema_long)
        self.data.dropna(inplace=True)
        self.data['returns'] = np.log(self.data.close / self.data.close.shift(1))
    
    def test(self):
        self.data["position"] = np.where(self.data["EMA_S"] > self.data["EMA_L"], 1, -1)
        self.data["returns"] = np.log(self.data.close.div(self.data.close.shift(1)))
        self.data["strategy"] = self.data.position.shift(1) * self.data.returns
        self.data.dropna(inplace=True)

 
    def calc_performance(self):
        self.data["strategy"] = self.data.position.shift(1) * self.data.returns
        self.data.dropna(inplace=True)
        self.data["cm_returns"] = self.data["returns"].cumsum().apply(np.exp)
        self.data["cm_strategy"] = self.data["strategy"].cumsum().apply(np.exp)
        out_perf = self.data["cm_strategy"].iloc[-1] - self.data["cm_returns"].iloc[-1]
        return round(out_perf, 2)
        

In [20]:
class IterativeBacktest(IterativeBase):
    def __init__(self):
        super().__init__()
        self.get_data()
    
    def test_ema_strategy(self, ema_short, ema_long):
        self.initiate_values(ema_short, ema_long)
        self.test()
        return self.calc_performance()

In [21]:
bc = IterativeBacktest()

In [22]:
ema_short = 5
ema_long = 26
bc.test_ema_strategy(ema_short, ema_long)

-2.1

In [24]:
ema_short = range(3, 30, 1)
ema_long = range(25,50, 1)

combinations = list(product(ema_short, ema_long))
results = []
i = 0
for comb in combinations:
    i+= 1
    print(*comb)
    results.append(bc.test_ema_strategy(*comb))
    clear_output(wait=True)
    print(i, ' of ', len(combinations))
#results

675  of  675


In [25]:
np.max(results)

-0.54

In [26]:
np.min(results)

-3.14

In [27]:
dfFinal = pd.DataFrame(data=combinations, columns=["EMA_S", "EMA_L"])
dfFinal["performance"] = results
dfFinal.tail()

,EMA_S,EMA_L,performance
670,29,45,-1.41
671,29,46,-1.61
672,29,47,-1.67
673,29,48,-1.57
674,29,49,-1.54


In [28]:
dfFinal.nlargest(10, "performance")

,EMA_S,EMA_L,performance
72,5,47,-0.54
44,4,44,-0.60
71,5,46,-0.61
231,12,31,-0.62
46,4,46,-0.63
43,4,43,-0.66
98,6,48,-0.66
45,4,45,-0.67
47,4,47,-0.67
229,12,29,-0.67


In [29]:
dfFinal.nsmallest(10, "performance")

,EMA_S,EMA_L,performance
550,25,25,-3.14
602,27,27,-3.14
576,26,26,-3.12
654,29,29,-3.11
628,28,28,-3.09
601,27,26,-2.79
575,26,25,-2.78
600,27,25,-2.76
652,29,27,-2.76
625,28,25,-2.73
